# Youtube Data Pipeline

In [90]:
import os

import pandas as pd
from html import unescape
from datetime import datetime
import isodate
import googleapiclient.discovery

from IPython.display import JSON

In [2]:
api_key = os.environ["YOUTUBE_API_KEY"]

## Channel Information

### Simple Code

In [3]:
api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=api_key
)

In [4]:
request = youtube.channels().list(
    part="snippet",#,contentDetails,statistics",
    forUsername="harsh1kumar"
    # id="UCueeXkuJezkCqu0YryvJnnQ,UCs8a-hjf6X4pa-O0orSoC8w"
)
response = request.execute()

In [5]:
JSON(response)

<IPython.core.display.JSON object>

### Function for channel information

In [6]:
def get_channel_info(youtube, channel_ids):
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(channel_ids)
    )
    response = request.execute()
    
    all_data = []
    for items in response["items"]:
        data = {
            "channel_name": items["snippet"]["title"],
            "view_count": items["statistics"]["viewCount"],
            "subscriber_count": items["statistics"]["subscriberCount"],
            "video_count": items["statistics"]["videoCount"],
            "channel_id": items["id"],
            "playlist_id": items["contentDetails"]["relatedPlaylists"]["uploads"],
        }

        all_data.append(data)

    return pd.DataFrame(all_data)

In [7]:
channel_ids = ["UCueeXkuJezkCqu0YryvJnnQ",   #@harsh1kumar
               "UCs8a-hjf6X4pa-O0orSoC8w",   #@amitvarma
               "UCJQJAI7IjbLcpsjWdSzYz0Q",   #@Thuvu5
              ]
channel_info = get_channel_info(youtube, channel_ids)
channel_info

channel_name view_count subscriber_count video_count  \
0            Harsh Kumar      76355              704          14   
1  Thu Vu data analytics    5505017           169000          75   
2              amitvarma     140585             5640          16   

                 channel_id               playlist_id  
0  UCueeXkuJezkCqu0YryvJnnQ  UUueeXkuJezkCqu0YryvJnnQ  
1  UCJQJAI7IjbLcpsjWdSzYz0Q  UUJQJAI7IjbLcpsjWdSzYz0Q  
2  UCs8a-hjf6X4pa-O0orSoC8w  UUs8a-hjf6X4pa-O0orSoC8w

## Playlist Information

### Simple Code

In [8]:
request = youtube.playlistItems().list(
    part="snippet,contentDetails",
    playlistId="UUJQJAI7IjbLcpsjWdSzYz0Q",
    maxResults=50
)
response = request.execute()

In [9]:
JSON(response)

<IPython.core.display.JSON object>

### Function for Playlist Information

In [10]:
def get_playlist_info(youtube, playlist_ids):
    
    all_data = []
    
    for pid in playlist_ids:
        next_page_token = ""

        while next_page_token is not None:
            request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                maxResults=50,
                playlistId=pid,
                pageToken = next_page_token
            )
            response = request.execute()

            for items in response["items"]:
                data = {
                    "title": items["snippet"]["title"],
                    "published_at": items["snippet"]["publishedAt"],
                    "channel_name": items["snippet"]["videoOwnerChannelTitle"],
                    "channel_id": items["snippet"]["channelId"],
                    "video_id": items["snippet"]["resourceId"]["videoId"]
                }
                all_data.append(data)
            
            next_page_token = response.get('nextPageToken')

    return pd.DataFrame(all_data)

In [11]:
playlist_info = get_playlist_info(youtube, channel_info["playlist_id"].to_list())
playlist_info.head()

title          published_at  \
0  MLFlow Tutorial | Hands-on | ML Tracking and S...  2023-04-17T10:30:03Z   
1  isort for sorting Python imports #shorts #pyth...  2023-03-26T13:41:44Z   
2  flake8 for linting in Python #shorts #python #...  2023-03-19T18:34:57Z   
3  pytest Tutorial: How to write tests in Python ...  2023-02-06T14:06:40Z   
4  Simple Neural Network using Tensorflow and Ker...  2021-10-18T06:32:21Z   

  channel_name                channel_id     video_id  
0  Harsh Kumar  UCueeXkuJezkCqu0YryvJnnQ  7Mv91hcxCCI  
1  Harsh Kumar  UCueeXkuJezkCqu0YryvJnnQ  rs8_I0sYGhw  
2  Harsh Kumar  UCueeXkuJezkCqu0YryvJnnQ  bURvz4g-XIg  
3  Harsh Kumar  UCueeXkuJezkCqu0YryvJnnQ  bhjaQssIXiw  
4  Harsh Kumar  UCueeXkuJezkCqu0YryvJnnQ  DqlPAWkkQC8

In [12]:
playlist_info.published_at = pd.to_datetime(playlist_info.published_at, format='%Y-%m-%dT%H:%M:%SZ')

In [13]:
playlist_info.shape

(105, 5)

In [14]:
playlist_info.groupby("channel_name", as_index=False).size()

channel_name  size
0            Harsh Kumar    14
1  Thu Vu data analytics    75
2              amitvarma    16

## Get video stats

### Simple Code

In [15]:
request = youtube.videos().list(
    part="contentDetails,snippet,statistics",
    id="7Mv91hcxCCI"
)
response = request.execute()

In [16]:
JSON(response)

<IPython.core.display.JSON object>

### Function for Video Information

In [17]:
def get_video_details(youtube, video_ids):
    request = youtube.videos().list(
        part="contentDetails,snippet,statistics",
        id=",".join(video_ids)
    )
    response = request.execute()
    
    all_data = []
    for items in response["items"]:
        data = {
            "video_id": items["id"],
            "title": items["snippet"]["title"],
            "published_at": items["snippet"]["publishedAt"],
            "duration": items["contentDetails"]["duration"],
            "view_count": items["statistics"]["viewCount"],
            "like_count": items["statistics"]["likeCount"],
            "comment_count": items["statistics"]["commentCount"],
        }

        all_data.append(data)

    return pd.DataFrame(all_data)

### Detail of latest video for each channel

In [18]:
playlist_info["recency_rank"] = playlist_info.groupby("channel_id")["published_at"].rank(method="first", ascending=False)
playlist_info.loc[playlist_info["recency_rank"]==1]

title        published_at  \
0   MLFlow Tutorial | Hands-on | ML Tracking and S... 2023-04-17 10:30:03   
14    Books to Learn about AI: You can’t miss these!! 2023-10-09 22:02:40   
89  The China Model is Broken | Episode 16 | Every... 2023-10-13 04:54:24   

             channel_name                channel_id     video_id  recency_rank  
0             Harsh Kumar  UCueeXkuJezkCqu0YryvJnnQ  7Mv91hcxCCI           1.0  
14  Thu Vu data analytics  UCJQJAI7IjbLcpsjWdSzYz0Q  uGynMyXCcg8           1.0  
89              amitvarma  UCs8a-hjf6X4pa-O0orSoC8w  eHX9sgBt1nE           1.0

In [19]:
latest_video_list = playlist_info.loc[playlist_info["recency_rank"]==1, "video_id"].to_list()

In [119]:
latest_video_details = get_video_details(youtube, latest_video_list)
latest_video_details

video_id                                              title  \
0  7Mv91hcxCCI  MLFlow Tutorial | Hands-on | ML Tracking and S...   
1  uGynMyXCcg8    Books to Learn about AI: You can’t miss these!!   
2  eHX9sgBt1nE  The China Model is Broken | Episode 16 | Every...   

           published_at    duration view_count like_count comment_count  
0  2023-04-17T10:30:03Z    PT13M48S        999         34             2  
1  2023-10-09T22:02:40Z       PT59S       3222        308             6  
2  2023-10-13T04:54:24Z  PT1H28M51S       3425        153            31

## Get Video Comments

### Simple Code

In [49]:
request = youtube.commentThreads().list(
    part="snippet,replies",
    maxResults=25,
    videoId="eHX9sgBt1nE"
)
response = request.execute()

In [50]:
JSON(response)

<IPython.core.display.JSON object>

### Function for Comment Information

In [29]:
def get_video_comments(youtube, video_ids):
    
    all_data = []
    for vid in video_ids:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            maxResults=100,
            videoId=vid,
        )
        response = request.execute()

        for items in response["items"]:
            data = {

                "comment_id": items["id"],
                "video_id": items["snippet"]["videoId"],
                "channel_id": items["snippet"]["channelId"],
                "published_at": items["snippet"]["topLevelComment"]["snippet"]["publishedAt"],
                "text_display": items["snippet"]["topLevelComment"]["snippet"]["textDisplay"],
                "author_name": items["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"],
                "like_count": items["snippet"]["topLevelComment"]["snippet"]["likeCount"],
            }

            all_data.append(data)

    return pd.DataFrame(all_data)

In [30]:
comment_details = get_video_comments(youtube, latest_video_list)
comment_details.head()

comment_id     video_id                channel_id  \
0  UgyIqOxKydbUPhSovAh4AaABAg  7Mv91hcxCCI  UCueeXkuJezkCqu0YryvJnnQ   
1  UgxWwT57bocTPxMWN6l4AaABAg  7Mv91hcxCCI  UCueeXkuJezkCqu0YryvJnnQ   
2  UgySNOjGeKJPxAF6e-B4AaABAg  uGynMyXCcg8  UCJQJAI7IjbLcpsjWdSzYz0Q   
3  UgyzdUk8Sf9XRTN73pd4AaABAg  uGynMyXCcg8  UCJQJAI7IjbLcpsjWdSzYz0Q   
4  UgxZ57mfjdA6YXfUE3x4AaABAg  uGynMyXCcg8  UCJQJAI7IjbLcpsjWdSzYz0Q   

           published_at                                       text_display  \
0  2023-06-22T02:59:00Z                        Great demo and walkthrough!   
1  2023-04-17T12:08:51Z                                             Thanks   
2  2023-10-10T08:31:49Z            Hoping to find some time for 1 of these   
3  2023-10-10T06:15:35Z  @Thuvu5 ma&#39;am can you do short overview of...   
4  2023-10-10T04:59:29Z                              such a great video...   

                      author_name  like_count  
0                Narasimha Murthy           0  
1           Sery christian renaud           1  
2                     Miro Krotký           1  
3                       Anaconda🐍           1  
4  Grow YouTube Views | Gain Fame           0

In [31]:
comment_details.shape

(26, 7)

In [37]:
comment_details.groupby("video_id", as_index=False).size()

video_id  size
0  7Mv91hcxCCI     2
1  eHX9sgBt1nE    20
2  uGynMyXCcg8     4

## Data Post Processing

Remove HTML character reference from string

In [75]:
comment_details["text_display"] = comment_details["text_display"].apply(unescape)

In [87]:
latest_video_details

video_id                                              title  \
0  7Mv91hcxCCI  MLFlow Tutorial | Hands-on | ML Tracking and S...   
1  uGynMyXCcg8    Books to Learn about AI: You can’t miss these!!   
2  eHX9sgBt1nE  The China Model is Broken | Episode 16 | Every...   

           published_at    duration view_count like_count comment_count  
0  2023-04-17T10:30:03Z    PT13M48S        998         34             2  
1  2023-10-09T22:02:40Z       PT59S       3218        308             6  
2  2023-10-13T04:54:24Z  PT1H28M51S       3406        153            31

Get proper duration

In [120]:
latest_video_details['duration_sec'] = latest_video_details['duration'].apply(lambda x: isodate.parse_duration(x))
latest_video_details['duration_sec'] = latest_video_details['duration_sec'].dt.total_seconds()

latest_video_details.drop('duration', axis=1, inplace=True)

Fix datatypes

In [130]:
channel_info.view_count = channel_info.view_count.astype(int)
channel_info.subscriber_count = channel_info.subscriber_count.astype(int)
channel_info.video_count = channel_info.video_count.astype(int)

In [138]:
latest_video_details.view_count = latest_video_details.view_count.astype(int)
latest_video_details.like_count = latest_video_details.like_count.astype(int)
latest_video_details.comment_count = latest_video_details.comment_count.astype(int)

latest_video_details.published_at = pd.to_datetime(latest_video_details.published_at, format='%Y-%m-%dT%H:%M:%SZ')

In [144]:
comment_details.like_count = comment_details.like_count.astype(int)

comment_details.published_at = pd.to_datetime(comment_details.published_at, format='%Y-%m-%dT%H:%M:%SZ')

## Push data to BQ

Before pushing, it is useful to add load_timestamp to the table

In [148]:
gcp_project_id = "wide-hexagon-397214"

Push channel details

In [149]:
channel_info["load_timestamp"] = datetime.now()

channel_info.to_gbq(destination_table='youtube_data.channel_info',
                     project_id=gcp_project_id,
                     if_exists='append')

Push video Details

In [150]:
latest_video_details["load_timestamp"] = datetime.now()

latest_video_details.to_gbq(destination_table='youtube_data.latest_video_details',
                            project_id=gcp_project_id,
                            if_exists='append')

Push comment details

In [151]:
comment_details["load_timestamp"] = datetime.now()

comment_details.to_gbq(destination_table='youtube_data.comment_details',
                       project_id=gcp_project_id,
                       if_exists='replace')